In [1]:
import numpy as np
import pandas as pd
import math

import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

def get_model_cv_prediction(model, X_data, y_target):
    neg_mse_scores = cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv = 5)
    rmse_scores = np.sqrt(-1 * neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)
    print('##### ', model.__class__.__name__, ' #####')
    print(' 5 교차 검증의 평균 RMSE : {0:.3f} '.format(avg_rmse))

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, ' model 시작 ')

    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('\t 폴드 세트: ', folder_counter, ' 시작 ')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        model.fit(X_tr, y_tr)

        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)

    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

    return train_fold_pred, test_pred_mean

In [2]:
train = pd.read_csv('../data/train.csv', index_col='id')
test = pd.read_csv('../data/test.csv', index_col='id')
submission = pd.read_csv('../data/sample_submission.csv', index_col='id')

In [12]:
train_upper = train[train['Y18'].isnull()]
train_upper = train_upper.drop(['X04', 'X14', 'X16', 'X19', 'X36', 'Y18'], axis=1)

In [13]:
train_upper.columns

Index(['X00', 'X01', 'X02', 'X03', 'X05', 'X06', 'X07', 'X08', 'X09', 'X10',
       'X11', 'X12', 'X13', 'X15', 'X17', 'X18', 'X20', 'X21', 'X22', 'X23',
       'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33',
       'X34', 'X35', 'X37', 'X38', 'X39', 'Y00', 'Y01', 'Y02', 'Y03', 'Y04',
       'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14',
       'Y15', 'Y16', 'Y17'],
      dtype='object')

In [21]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso

xgbr = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

lgbmr = LGBMRegressor(colsample_bylevel= 0.5, colsample_bytree= 0.7, max_depth= 3, n_estimators= 1500)

lasso_reg = Lasso(alpha= 0.001)

In [17]:
columnList = train_upper.columns
target_var_list = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17']

input_var = list(filter(lambda x: target_var_list.count(x) == 0, columnList))
input_var

['X00',
 'X01',
 'X02',
 'X03',
 'X05',
 'X06',
 'X07',
 'X08',
 'X09',
 'X10',
 'X11',
 'X12',
 'X13',
 'X15',
 'X17',
 'X18',
 'X20',
 'X21',
 'X22',
 'X23',
 'X24',
 'X25',
 'X26',
 'X27',
 'X28',
 'X29',
 'X30',
 'X31',
 'X32',
 'X33',
 'X34',
 'X35',
 'X37',
 'X38',
 'X39']

In [18]:
X_features = train_upper[input_var]
y_target = train_upper['Y00']

In [19]:
models = [xgbr, lgbmr]

for model in models:
    get_model_cv_prediction(model, X_features, y_target)

[19:33:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:34:02] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:34:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:34:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:34:28] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
#####  XGBRegressor  #####
 5 교차 검증의 평균 RMSE : 0.576 
#####  LGBMRegressor  #####
 5 교차 검증의 평균 RMSE : 0.594 


In [35]:
def three_model_auto(X_train_n, X_test_n, y_train_n):
    lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
    xgb_train, xgb_test = get_stacking_base_datasets(xgbr, X_train_n, y_train_n, X_test_n, 5)
    lgbm_train, lgbm_test = get_stacking_base_datasets(lgbmr, X_train_n, y_train_n, X_test_n, 5)

    Stack_final_X_train = np.concatenate((lasso_train,xgb_train,lgbm_train), axis=1)
    Stack_final_X_test = np.concatenate((lasso_test,xgb_test,lgbm_test), axis=1)

    return Stack_final_X_train, Stack_final_X_test


In [36]:
from sklearn.metrics import mean_squared_error

def stack_auto(X_features, y_target):
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=0)

    X_train_n = X_train.values
    X_test_n = X_test.values
    y_train_n = y_train.values

    Stack_final_X_train, Stack_final_X_test = three_model_auto(X_train_n, X_test_n, y_train_n)

    meta_model_lasso = Lasso(alpha=0.0005)

    meta_model_lasso.fit(Stack_final_X_train, y_train)
    final = meta_model_lasso.predict(Stack_final_X_test)
    mse = mean_squared_error(y_test, final)
    rmse = np.sqrt(mse)
    print('스태킹 회귀 모델 최종 RMSE :: ',rmse)

    return meta_model_lasso

In [28]:
trainNotNull = train[train['Y18'].notnull()]
trainNotNull = trainNotNull.drop(['X04', 'X14', 'X16', 'X19', 'X36'], axis=1)
trainNotNull

,X00,X01,X02,X03,X05,X06,X07,X08,X09,X10,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
4320,19.3,987.7,0.9,2.2,1007.7,988.0,20.8,1007.8,1007.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4321,19.0,987.6,1.9,2.2,1007.7,988.0,20.6,1007.8,1007.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4322,19.1,987.6,2.0,1.4,1007.8,988.1,20.5,1007.9,1007.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4323,19.2,987.7,1.8,1.5,1007.9,988.1,20.5,1007.9,1007.5,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4324,19.2,987.8,1.4,1.4,1007.9,988.1,20.5,1007.8,1007.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4325,19.0,987.9,1.5,1.2,1007.8,988.1,20.4,1008.0,1007.7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4326,19.1,987.9,1.2,1.0,1007.8,988.1,20.3,1008.0,1007.7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4327,19.3,987.8,0.7,0.6,1007.8,988.1,20.4,1008.0,1007.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5
4328,19.1,987.7,2.7,0.1,1007.7,988.1,20.4,1008.0,1007.5,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5


In [42]:
# 로직 테스트용

# target_col = 'Y00'
# y_target = train_upper[target_col]
# model = stack_auto(X_features, y_target)

# X_train_n = train_upper[input_var].values
# X_test_n = trainNotNull[input_var].values
# y_train_n = train_upper[target_col].values

# final_X_train, final_X_test = three_model_auto(X_train_n, X_test_n, y_train_n)

# pred = model.predict(final_X_test)
# trainNotNull[target_col] = pred

Lasso  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[20:21:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[20:21:58] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[20:22:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[20:22:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  4  시작 
[20:22:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated

In [45]:
input_var

for target_col in target_var_list:
    print(target_col)
    y_target = train_upper[target_col]
    model = stack_auto(X_features, y_target)

    X_train_n = train_upper[input_var].values
    X_test_n = trainNotNull[input_var].values
    y_train_n = train_upper[target_col].values

    final_X_train, final_X_test = three_model_auto(X_train_n, X_test_n, y_train_n)

    pred = model.predict(final_X_test)
    trainNotNull[target_col] = pred

ace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGBMRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
스태킹 회귀 모델 최종 RMSE ::  0.8920736763526722
Lasso  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[20:39:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[20:39:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[20:40:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[20:40:15] WARNING: C:/Jenkins/works

In [46]:
trainNotNull

,X00,X01,X02,X03,X05,X06,X07,X08,X09,X10,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
4320,19.3,987.7,0.9,2.2,1007.7,988.0,20.8,1007.8,1007.4,0.0,...,18.439546,19.475234,18.377823,18.030998,19.343037,19.052722,18.622477,18.972583,18.515445,20.5
4321,19.0,987.6,1.9,2.2,1007.7,988.0,20.6,1007.8,1007.4,0.0,...,19.067864,19.505837,18.562473,18.979274,19.410064,19.023436,18.951382,19.165318,19.101177,20.5
4322,19.1,987.6,2.0,1.4,1007.8,988.1,20.5,1007.9,1007.4,0.0,...,18.804229,19.200391,18.349829,19.293100,19.426396,18.889947,18.822606,19.021895,18.664658,20.5
4323,19.2,987.7,1.8,1.5,1007.9,988.1,20.5,1007.9,1007.5,0.0,...,18.699149,19.048839,18.297403,19.154612,19.377473,18.864614,18.748349,18.914470,18.793899,20.5
4324,19.2,987.8,1.4,1.4,1007.9,988.1,20.5,1007.8,1007.6,0.0,...,18.648370,19.213708,18.374789,19.003732,19.293195,18.952169,18.679577,18.715628,18.952002,20.5
4325,19.0,987.9,1.5,1.2,1007.8,988.1,20.4,1008.0,1007.7,0.0,...,18.826371,19.186293,18.543343,19.636036,19.312308,18.910992,18.974639,19.055975,19.090721,20.5
4326,19.1,987.9,1.2,1.0,1007.8,988.1,20.3,1008.0,1007.7,0.0,...,19.112447,19.374088,18.455125,19.543015,19.326520,18.969819,19.181866,19.037785,19.062968,20.5
4327,19.3,987.8,0.7,0.6,1007.8,988.1,20.4,1008.0,1007.6,0.0,...,19.091155,19.391910,18.355611,19.621988,19.367680,19.010157,19.369282,19.118489,19.096539,20.5
4328,19.1,987.7,2.7,0.1,1007.7,988.1,20.4,1008.0,1007.5,0.0,...,19.185957,19.423751,18.558444,19.704696,19.500293,19.040693,19.326081,19.146577,19.045105,20.5


In [54]:
# test

trainNotNull_X_features = trainNotNull[input_var]

for target_col in target_var_list:
    print(target_col)
    trainNotNull_y_target = trainNotNull[target_col]
    model = stack_auto(trainNotNull_X_features, trainNotNull_y_target)

    X_train_n = trainNotNull[input_var].values
    X_test_n = test[input_var].values
    y_train_n = trainNotNull[target_col].values

    final_X_train, final_X_test = three_model_auto(X_train_n, X_test_n, y_train_n)

    pred = model.predict(final_X_test)
    test[target_col] = pred



c/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGBMRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
스태킹 회귀 모델 최종 RMSE ::  0.5521895964198585
Lasso  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[21:09:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[21:09:56] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[21:09:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[21:09:59] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/s

In [55]:
test

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,Y08,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17
id,,,,,,,,,,,,,,,,,,,,,
4752,18.7,987.4,1.5,1.3,0.0,1006.8,987.7,21.1,1007.5,1007.2,...,19.648454,19.128743,20.529617,19.521364,18.709853,19.422173,19.076212,19.023751,18.713419,18.567003
4753,18.8,987.4,1.1,1.2,0.0,1006.7,987.7,21.2,1007.5,1007.2,...,19.551806,19.066508,20.406179,19.432163,18.568011,19.400376,19.118552,18.989041,18.656124,18.605782
4754,19.0,987.4,1.3,1.2,0.0,1006.6,987.6,21.2,1007.4,1007.2,...,19.697412,19.169566,20.338671,19.479480,18.574304,19.329780,19.074501,18.939696,18.835600,18.669479
4755,18.7,987.4,2.3,0.8,0.0,1006.6,987.6,21.1,1007.4,1007.2,...,19.682920,18.977581,20.083214,19.274518,18.473481,19.361711,19.081058,18.779703,18.713096,18.476679
4756,18.4,987.4,1.1,0.7,0.0,1006.7,987.7,20.9,1007.5,1007.2,...,19.679193,18.801915,19.978217,19.200149,18.373916,19.273154,18.928096,18.871596,18.595590,18.517775
4757,18.6,987.4,1.3,0.0,0.0,1006.8,987.7,20.7,1007.5,1007.2,...,19.496005,18.728267,19.811497,18.925941,18.583371,19.034339,18.798871,18.817567,18.462201,18.516697
4758,18.6,987.3,1.0,0.8,0.0,1006.7,987.6,20.6,1007.4,1007.1,...,19.559173,18.858255,19.887572,19.015539,18.455909,19.127768,18.847929,18.936116,18.642331,18.548722
4759,18.7,987.5,1.5,0.8,0.0,1006.7,987.5,20.5,1007.4,1007.3,...,19.457070,18.555981,19.650603,18.656911,18.401225,18.949259,18.518635,18.826383,18.615573,18.433475
4760,18.5,987.5,1.2,1.6,0.0,1006.8,987.4,20.4,1007.3,1007.4,...,19.371262,18.531993,19.675686,18.634483,18.393927,18.852802,18.490774,18.804216,18.537353,18.328907


In [56]:
trainNotNull

input_var_y = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17']
target_var_y = ['Y18']

trainNotNull_features_y = trainNotNull[input_var_y]
trainNotNull_y_target = trainNotNull[target_var_y]
modelBB = stack_auto(trainNotNull_features_y, trainNotNull_y_target)

trainNotNull_X_train_n = trainNotNull[input_var_y].values
trainNotNull_X_test_n = test[input_var_y].values
trainNotNull_y_train_n = trainNotNull[target_var_y].values

trainNotNull_final_X_train, trainNotNull_final_X_test = three_model_auto(trainNotNull_X_train_n, trainNotNull_X_test_n, trainNotNull_y_train_n)

pred = modelBB.predict(trainNotNull_final_X_test)
pred


Lasso  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[21:14:30] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[21:14:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[21:14:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[21:14:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  4  시작 
[21:14:32] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated

array([20.69409375, 20.69581551, 20.68463488, ..., 29.67789307,
       29.67768438, 30.55890956])

In [57]:
len(pred)

11520

In [58]:
outputDf = pd.DataFrame()
outputDf['id'] = test.index
outputDf['Y18'] = pred
outputDf.to_csv('submission.csv', index=False)